In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# keeps the plots in one place. calls image as static pngs
# %matplotlib inline 
import matplotlib.pyplot as plt # side-stepping mpl backend
import matplotlib.gridspec as gridspec # subplots
import mpld3 as mpl

#Import models from scikit learn module:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics

In [5]:
df = pd.read_csv("data.csv",header = 0)


In [14]:
print(pd.io.sql.get_schema(df, "breast_cancer_detection_raw"))

CREATE TABLE "breast_cancer_detection_raw" (
"id" INTEGER,
  "diagnosis" TEXT,
  "radius_mean" REAL,
  "texture_mean" REAL,
  "perimeter_mean" REAL,
  "area_mean" REAL,
  "smoothness_mean" REAL,
  "compactness_mean" REAL,
  "concavity_mean" REAL,
  "concave points_mean" REAL,
  "symmetry_mean" REAL,
  "fractal_dimension_mean" REAL,
  "radius_se" REAL,
  "texture_se" REAL,
  "perimeter_se" REAL,
  "area_se" REAL,
  "smoothness_se" REAL,
  "compactness_se" REAL,
  "concavity_se" REAL,
  "concave points_se" REAL,
  "symmetry_se" REAL,
  "fractal_dimension_se" REAL,
  "radius_worst" REAL,
  "texture_worst" REAL,
  "perimeter_worst" REAL,
  "area_worst" REAL,
  "smoothness_worst" REAL,
  "compactness_worst" REAL,
  "concavity_worst" REAL,
  "concave points_worst" REAL,
  "symmetry_worst" REAL,
  "fractal_dimension_worst" REAL,
  "Unnamed: 32" REAL
)


In [30]:
df.drop('id',axis=1,inplace=True)
df.drop('Unnamed: 32',axis=1,inplace=True)

In [31]:
df['diagnosis'] = df['diagnosis'].map({'M':1,'B':0})
features_mean=list(df.columns[1:11])



In [32]:
#Generic function for making a classification model and accessing the performance. 
# From AnalyticsVidhya tutorial
def classification_model(model, data, predictors, outcome):
  #Fit the model:
  model.fit(data[predictors],data[outcome])
  
  #Make predictions on training set:
  predictions = model.predict(data[predictors])
  
  #Print accuracy
  accuracy = metrics.accuracy_score(predictions,data[outcome])
  print("Accuracy : %s" % "{0:.3%}".format(accuracy))

  #Perform k-fold cross-validation with 5 folds
  kf = KFold(n_splits=5)
  error = []
  for train, test in kf.split(data):
    # Filter training data
    train_predictors = (data[predictors].iloc[train,:])
    
    # The target we're using to train the algorithm.
    train_target = data[outcome].iloc[train]
    
    # Training the algorithm using the predictors and target.
    model.fit(train_predictors, train_target)

    error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    
    print("Cross-Validation Score : %s" % "{0:.3%}".format(np.mean(error)))
    
  #Fit the model again so that it can be refered outside the function:
  model.fit(data[predictors],data[outcome])

In [33]:
traindf, testdf = train_test_split(df, test_size = 0.3)


In [34]:
predictor_var = ['radius_mean','perimeter_mean','area_mean','compactness_mean','concave points_mean']
outcome_var='diagnosis'
model=LogisticRegression()
classification_model(model,traindf,predictor_var,outcome_var)

Accuracy : 88.693%
Cross-Validation Score : 87.500%
Cross-Validation Score : 91.250%
Cross-Validation Score : 88.333%
Cross-Validation Score : 88.085%
Cross-Validation Score : 88.696%


In [38]:
predictor_var = features_mean
model = RandomForestClassifier(n_estimators=100,min_samples_split=25, max_depth=7, max_features=2)
classification_model(model, traindf,predictor_var,outcome_var)

Accuracy : 95.477%
Cross-Validation Score : 95.000%
Cross-Validation Score : 93.750%
Cross-Validation Score : 91.667%
Cross-Validation Score : 92.168%
Cross-Validation Score : 92.975%


In [50]:
output = model.predict(traindf[features_mean])

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,

In [47]:
class A:
    def __init__(self):
        self.a = 1
    
    def testb(self):
        print(self.a)
    def testa(self):
        return testb()

In [53]:
traindf.columns.h

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [62]:
traindf.iloc[0]

diagnosis                    0.000000
radius_mean                 13.270000
texture_mean                17.020000
perimeter_mean              84.550000
area_mean                  546.400000
smoothness_mean              0.084450
compactness_mean             0.049940
concavity_mean               0.035540
concave points_mean          0.024560
symmetry_mean                0.149600
fractal_dimension_mean       0.056740
radius_se                    0.292700
texture_se                   0.890700
perimeter_se                 2.044000
area_se                     24.680000
smoothness_se                0.006032
compactness_se               0.011040
concavity_se                 0.022590
concave points_se            0.009057
symmetry_se                  0.014820
fractal_dimension_se         0.002496
radius_worst                15.140000
texture_worst               23.600000
perimeter_worst             98.840000
area_worst                 708.800000
smoothness_worst             0.127600
compactness_

In [65]:
input_vector = pd.DataFrame(
        [{"diagnosis": 1,
          'radius_mean': 13.270000,
          "texture_mean": 17.020000,
          "perimter_mean": 84.550000,
          "area_mean": 546.400000,
          "smoothness_mean": 0.084450,
          "compactness_mean": 0.049940,
          "concavity_mean": 0.035540,
          "concave points_mean": 0.024560,
          "symmetry_mean": 0.149600,
          "fractal_dimension_mean": 0.056740,
          "id": 1,
          "Unnamed: 32": 2
          
          
        }], features_mean
    )

In [69]:
features_mean += ["id", "Unnamed: 32"]

['radius_mean',
 'texture_mean',
 'perimeter_mean',
 'area_mean',
 'smoothness_mean',
 'compactness_mean',
 'concavity_mean',
 'concave points_mean',
 'symmetry_mean',
 'fractal_dimension_mean',
 'id',
 'Unnamed: 32']

In [66]:
input_vector

,diagnosis,radius_mean,texture_mean,perimter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
radius_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
texture_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
perimeter_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
area_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
smoothness_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
compactness_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
concavity_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
concave points_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
symmetry_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674
fractal_dimension_mean,1,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674


KeyError: "['a'] not found in axis"

In [4]:
features = ['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean']


input_vector = pd.DataFrame(
        data=[[13.270000,17.020000,84.550000,546.400000,0.084450,0.049940,0.035540,0.024560,0.149600,0.056740]], columns=features
    )
input_vector.set_axis(['id'])

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean
id,13.27,17.02,84.55,546.4,0.08445,0.04994,0.03554,0.02456,0.1496,0.05674


In [5]:
from qwak_inference import RealTimeClient


In [6]:
client = RealTimeClient(model_id="cancer_detection")
output = client.predict(input_vector)

In [7]:
output

['diagnosis']

In [83]:
input_vector.drop('id',axis=1,inplace=True)


In [97]:
test = pd.DataFrame(data={"a": [1,2,3], "b": ["a","b","😀"]})

In [98]:
test

,a,b
0,1,a
1,2,b
2,3,😀


In [ ]:
build features ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']


In [ ]:
predict features ['perimeter_mean', 'smoothness_mean', 'concave points_mean', 'compactness_mean', 'concavity_mean', 'fractal_dimension_mean', 'symmetry_mean', 'texture_mean', 'area_mean', 'radius_mean']


In [ ]:
export QWAK_TOKEN="eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlJuZjAySWIzSEJzTVFUWWJjblNwZSJ9.eyJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9xd2FrLWVudmlyb25tbmV0LWlkIjpbeyJkZWZhdWx0Ijp0cnVlLCJkYXRhX2FwaSI6IiIsIm1vZGVsc19hcGkiOiJtb2RlbHMuaHVkc29uLXRlc3QucXdhay5haSIsImJyYW5kX2xvZ29faW1hZ2VfcGF0aCI6IiIsImdycGNfYXBpIjoiZ3JwYy5odWRzb24tdGVzdC5xd2FrLmFpIiwiYnVja2V0IjoicXdha3N0YWNrY2xvdWQtbGFrZWRiZGRkNmY2LTk1MXV3c244ZThjbSIsImlkIjoiMDAwMDg2MjktN2ZjZC00MzBlLWFjZjYtOTVlYjBiZDkyNWFhIiwiYWxpYXMiOiJodWRzb24tdGVzdCJ9XSwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvcXdhay1wYXJ0bmVyLWlkIjoiMDAwMDg2MjktN2ZjZC00MzBlLWFjZjYtOTVlYjBiZDkyNWFhIiwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvcXdhay11c2VyLWlkIjoiaHVkc29uemJ1emJ5QGdtYWlsLmNvbSIsImh0dHBzOi8vYXV0aC10b2tlbi5xd2FrLmFpL2RhdGEtYXBpIjoiIiwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvZ3JwYy1hcGkiOiJncnBjLmh1ZHNvbi10ZXN0LnF3YWsuYWkiLCJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9tb2RlbHMtYXBpIjoibW9kZWxzLmh1ZHNvbi10ZXN0LnF3YWsuYWkiLCJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9xd2FrLWJ1Y2tldCI6InF3YWtzdGFja2Nsb3VkLWxha2VkYmRkZDZmNi05NTF1d3NuOGU4Y20iLCJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9idWNrZXQiOiJxd2Frc3RhY2tjbG91ZC1sYWtlZGJkZGQ2ZjYtOTUxdXdzbjhlOGNtIiwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvYmktYXBpLWtleSI6IiIsImh0dHBzOi8vYXV0aC10b2tlbi5xd2FrLmFpL3F3YWtfYWNjb3VudHMiOnsiMDAwMDFhNTctZmFhNi00MjU1LWIyZTEtY2EyMWE3MmM1NWFjIjp7ImlkIjoiMDAwMDFhNTctZmFhNi00MjU1LWIyZTEtY2EyMWE3MmM1NWFjIiwibmFtZSI6Imh1ZHNvbi10ZXN0IiwiZGVmYXVsdF9lbnZpcm9ubWVudF9pZCI6IjAwMDA4NjI5LTdmY2QtNDMwZS1hY2Y2LTk1ZWIwYmQ5MjVhYSIsInF3YWtfZW52aXJvbm1lbnRzIjp7IjAwMDA4NjI5LTdmY2QtNDMwZS1hY2Y2LTk1ZWIwYmQ5MjVhYSI6eyJpZCI6IjAwMDA4NjI5LTdmY2QtNDMwZS1hY2Y2LTk1ZWIwYmQ5MjVhYSIsImhvc3RzIjp7ImFpIjp7InF3YWsiOnsiY2xvdWQiOnsicHJpdmF0ZWxpbmsiOnsiZ3JwYyI6e319fSwiaHVkc29uLXRlc3QiOnsibW9kZWxzIjp7fSwiZ3JwYyI6e319fX19LCJhbGlhcyI6Imh1ZHNvbi10ZXN0In19fX0sImh0dHBzOi8vYXV0aC10b2tlbi5xd2FrLmFpL3F3YWtfdXNlcl9raW5kIjoiU0lNUExFX1VTRVIiLCJpc3MiOiJodHRwczovL2F1dGgucXdhay5haS8iLCJzdWIiOiJnb29nbGUtb2F1dGgyfDEwMTQwMzg2NTcxNTIwODIxNjA5MiIsImF1ZCI6WyJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS8iLCJodHRwczovL2Rldi1xd2FrLnVzLmF1dGgwLmNvbS91c2VyaW5mbyJdLCJpYXQiOjE2OTI3MTI1MzQsImV4cCI6MTY5Mjc5ODkzNCwiYXpwIjoidERScGV1SlFmeXZKRzZNMGVVWDJIancyUlhlWjFlSEsiLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIG9mZmxpbmVfYWNjZXNzIn0.P0bOJMwHVzCIHDHWttLMfDv3nh4yF8A-pHmZPGtxpb0WjwO5CVbCPYWAZFogIhsd8UJ2TxM4UIh6sG_gXo31XYzs60tF9TsA50uuDVxiX1QqbjPaMsQUIbPq_FqaFXGfygkr83oQbV_PXx2rpzm0AV6FBVzujh9xXqUoTtTMHL4xfIXJqpZvdWzbAFlk3rMIdVhmdt1c2dFeP-QmqhOVdonPQcAwa6C4LhcwMohujy1c6HHl88dqngCZMEu5I_SOdlryqKy-Y5deuS3_s4z3fqD8d-xfg8stuNF2MSnBj8ECVvrvU4pT-JG4weRoTP5WFBabhQXc_y-MwQOnLor5hA"

curl --location --request POST 'https://models.hudson-test.qwak.ai/v1/cancer_detection/predict' \
--header 'Content-Type: application/json' \
--header 'Authorization: Bearer '$QWAK_TOKEN'' \
--data '{"columns":["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","symmetry_mean","fractal_dimension_mean"],"index":[0],"data":[[13.270000,17.020000,84.550000,546.400000,0.084450,0.049940,0.035540,0.149600,0.056740]]}'


# from qwak.feature_store.data_sources import SnowflakeSource
from qwak.clients.secret_service import SecretServiceClient




secret_service = SecretServiceClient()


snowflake_source = SnowflakeSource(
    name='snowflake_source',
    description='a snowflake table for breast cancer detection data',
    date_created_column='ETL_TIME',
    host=secret_service.get_secret("snowflake-host"),
    username_secret_name="snowflake-user", # use secret service
    password_secret_name="snowflake-password", # use secret service
    database='QWAK_DB',
    schema='BREAST_CANCER_DETECTION',
    warehouse='COMPUTE_WH',
    table="breast_cancer_detection_raw"
)

snowflake_source.get_sample()

In [18]:
secret_service.get_secret("snowflake-host")

'taa51111.snowflakecomputing.com'

In [15]:
snowflake_source.username_secret_name

'qwakuser'

In [22]:
key_to_features = {'id': [
    "breast-cancer-ingestion.radius_mean",
    "breast-cancer-ingestion.texture_mean",
    "breast-cancer-ingestion.perimeter_mean",
    "breast-cancer-ingestion.area_mean",
    "breast-cancer-ingestion.smoothness_mean",
    "breast-cancer-ingestion.compactness_mean",
    "breast-cancer-ingestion.concavity_mean",
    "breast-cancer-ingestion.symmetry_mean",
    "breast-cancer-ingestion.fractal_dimension_mean",
    "breast-cancer-ingestion.diagnosis"
    ]
}

In [23]:
df = pd.DataFrame(
    columns=[                 "id"                ],
    data   =[[ 1]]
)

In [ ]:
import datetime
end_date = datetime.datetime.now()
start_date = datetime.datetime(2023,8,22,0,0,0)
train_df = offline_client.get_feature_range_values(
            entity_key_to_features=key_to_features,
            start_date=start_date,
            end_date=end_date,
        )


In [ ]:
export QWAK_TOKEN="eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IlJuZjAySWIzSEJzTVFUWWJjblNwZSJ9.eyJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9xd2FrLWVudmlyb25tbmV0LWlkIjpbeyJkZWZhdWx0Ijp0cnVlLCJkYXRhX2FwaSI6IiIsIm1vZGVsc19hcGkiOiJtb2RlbHMuaHVkc29uLXRlc3QucXdhay5haSIsImJyYW5kX2xvZ29faW1hZ2VfcGF0aCI6IiIsImdycGNfYXBpIjoiZ3JwYy5odWRzb24tdGVzdC5xd2FrLmFpIiwiYnVja2V0IjoicXdha3N0YWNrY2xvdWQtbGFrZWRiZGRkNmY2LTk1MXV3c244ZThjbSIsImlkIjoiMDAwMDg2MjktN2ZjZC00MzBlLWFjZjYtOTVlYjBiZDkyNWFhIiwiYWxpYXMiOiJodWRzb24tdGVzdCJ9XSwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvcXdhay1wYXJ0bmVyLWlkIjoiMDAwMDg2MjktN2ZjZC00MzBlLWFjZjYtOTVlYjBiZDkyNWFhIiwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvcXdhay11c2VyLWlkIjoiaHVkc29uemJ1emJ5QGdtYWlsLmNvbSIsImh0dHBzOi8vYXV0aC10b2tlbi5xd2FrLmFpL2RhdGEtYXBpIjoiIiwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvZ3JwYy1hcGkiOiJncnBjLmh1ZHNvbi10ZXN0LnF3YWsuYWkiLCJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9tb2RlbHMtYXBpIjoibW9kZWxzLmh1ZHNvbi10ZXN0LnF3YWsuYWkiLCJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9xd2FrLWJ1Y2tldCI6InF3YWtzdGFja2Nsb3VkLWxha2VkYmRkZDZmNi05NTF1d3NuOGU4Y20iLCJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS9idWNrZXQiOiJxd2Frc3RhY2tjbG91ZC1sYWtlZGJkZGQ2ZjYtOTUxdXdzbjhlOGNtIiwiaHR0cHM6Ly9hdXRoLXRva2VuLnF3YWsuYWkvYmktYXBpLWtleSI6IiIsImh0dHBzOi8vYXV0aC10b2tlbi5xd2FrLmFpL3F3YWtfYWNjb3VudHMiOnsiMDAwMDFhNTctZmFhNi00MjU1LWIyZTEtY2EyMWE3MmM1NWFjIjp7ImlkIjoiMDAwMDFhNTctZmFhNi00MjU1LWIyZTEtY2EyMWE3MmM1NWFjIiwibmFtZSI6Imh1ZHNvbi10ZXN0IiwiZGVmYXVsdF9lbnZpcm9ubWVudF9pZCI6IjAwMDA4NjI5LTdmY2QtNDMwZS1hY2Y2LTk1ZWIwYmQ5MjVhYSIsInF3YWtfZW52aXJvbm1lbnRzIjp7IjAwMDA4NjI5LTdmY2QtNDMwZS1hY2Y2LTk1ZWIwYmQ5MjVhYSI6eyJpZCI6IjAwMDA4NjI5LTdmY2QtNDMwZS1hY2Y2LTk1ZWIwYmQ5MjVhYSIsImhvc3RzIjp7ImFpIjp7InF3YWsiOnsiY2xvdWQiOnsicHJpdmF0ZWxpbmsiOnsiZ3JwYyI6e319fSwiaHVkc29uLXRlc3QiOnsibW9kZWxzIjp7fSwiZ3JwYyI6e319fX19LCJhbGlhcyI6Imh1ZHNvbi10ZXN0In19fX0sImh0dHBzOi8vYXV0aC10b2tlbi5xd2FrLmFpL3F3YWtfdXNlcl9raW5kIjoiU0lNUExFX1VTRVIiLCJpc3MiOiJodHRwczovL2F1dGgucXdhay5haS8iLCJzdWIiOiJnb29nbGUtb2F1dGgyfDEwMTQwMzg2NTcxNTIwODIxNjA5MiIsImF1ZCI6WyJodHRwczovL2F1dGgtdG9rZW4ucXdhay5haS8iLCJodHRwczovL2Rldi1xd2FrLnVzLmF1dGgwLmNvbS91c2VyaW5mbyJdLCJpYXQiOjE2OTI3OTcyMTIsImV4cCI6MTY5Mjg4MzYxMiwiYXpwIjoidERScGV1SlFmeXZKRzZNMGVVWDJIancyUlhlWjFlSEsiLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGVtYWlsIG9mZmxpbmVfYWNjZXNzIn0.BuXvwZQqIM3F1x0NqKideEdUs_25DwtFGtJhSwY_81Re4Ty8IV33HiUMQUE48l9rVNZCS9_h-PJvAvahPqUsHkDKXXKbD8anBiJ4FEc4bmOhvcjTgFvrxsqz4Atidrqyr7TLszt_voggd91m6_ldcdQ-OIkKWVsuSENG5kaFTcvmz-BVlygmZR4CSLQuCgLksOsf2vROhgvr-qjFmRhMpGJNbGMEYtGmGqCPstKuMjJXpoRIJNdrze6x1BoNDIqu24aaXIPp14GWGvDNnTj7SvfQsHlKPGblbxbW9hIMEZ-q7AMjrOTzZnkCZj12V8as2B-NGji7lEyaG_H-sbXjdw"

curl --location --request POST 'https://models.hudson-test.qwak.ai/v1/cancer_detection/predict' \
--header 'Content-Type: application/json' \
--header 'Authorization: Bearer '$QWAK_TOKEN'' \
--data '{"columns":["radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","symmetry_mean","fractal_dimension_mean"],"index":[0],"data":[[17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.2419,0.07871]]}'